In [1]:
import findspark
findspark.init()

In [2]:
# !pip install findspark

In [3]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [4]:
sc = SparkContext()

In [5]:
ssc = StreamingContext(sc,10)
sqlContext = SQLContext(sc)

In [6]:
socket_stream = ssc.socketTextStream("127.0.0.1", 5559)

In [7]:
lines = socket_stream.window(20)

In [8]:
from collections import namedtuple
fields = ("tag", "count")
Tweet = namedtuple('Tweet', fields)

In [9]:
( lines.flatMap( lambda text: text.split(" ")) \
    .filter(lambda word: word.lower().startswith("#")) \
    .map(lambda word: (word.lower(),1) \
    .reduceByKey(lambda a,b: a + b) \
    .map(lambda rec: Tweet( rec[0], rec[1])) \
    .foreachRDD(lambda rdd: rdd.toDF().sort( desc("count")) \
    .limit(10).registerTempTable("tweets")))
)

In [10]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
import pandas
%matplotlib inline

In [11]:
# !pip install matplotlib seaborn

In [12]:
ssc.start()

IllegalArgumentException: u'requirement failed: No output operations registered, so nothing to execute'

In [ ]:
count = 0 
while count < 10:
    
    time.sleep(3)
    top_10_tweets = sqlContext.sql('Select tag, count from tweets')
    top_10_df = top_10_tweets.toPandas()
    display.clear_output(wait=True)
    plt.figure(figsize = (10,8))
    sns.barplot(x="count", y="tag",d ata= top_10_df)
    plt.show()
    count = count + 1

In [ ]:
ssc.stop()